In [ ]:
KAFKA_TOPIC  = "IZkafkatopic.testdb.testiz"
KAFKA_BOOTSTRAP_SERVERS_CONS = hname2
n_secs=5

customparksession.sparkContext.setLogLevel("WARN")
#sc=customparksession.sparkContext
#ssc = StreamingContext(sc,n_secs)
ssc = StreamingContext(customparksession.sparkContext,n_secs)

fromOffset = {topicPartion: 0}

directkafkastream = KafkaUtils.createDirectStream(ssc,[KAFKA_TOPIC], 
                                            {'bootstrap.servers':KAFKA_BOOTSTRAP_SERVERS_CONS,
                                             #'startingoffsets':'earliest',
                                             #'auto.offset.reset':'smallest',
                                             #"max.poll.records":'100'
                                             #,'fetch.message.max.bytes':'15728640'
                                            }
                                            ,fromOffsets=fromOffset)

#Defined the HBase catalog

#catalog = {
#"table":{"namespace":"default", "name":"iverson_doing_sillythings"},\
#    "rowkey":"key",\
#        "columns":{\
#        "col0":{"cf":"rowkey", "col":"key", "type":"string"},\
#        "col1":{"cf":"cf1", "col":"col1", "type":"string"},\
#        "col2":{"cf":"cf1", "col":"col2", "type":"string"},\
#        "col3":{"cf":"cf1", "col":"col3", "type":"float"},\
#        "col4":{"cf":"cf1", "col":"col4", "type":"float"}\
#                  }\
#            }

catalog = ''.join("""{
  "table":{"namespace":"iverson", "name":"iversondoingsometingsilly3"},
  "rowkey":"key",
  "columns":{
  "col0":{"cf":"rowkey", "col":"key", "type":"string"},
  "col1":{"cf":"info", "col":"basicinfo", "type":"string"}
  }
  }""".split())

data_source_format = 'org.apache.spark.sql.execution.datasources.hbase'

def Iversondoingsomethingsilly():
    dump_rdd = directkafkastream.map(lambda m: json.loads(m))
    load_rdd = dump_rdd.map(lambda x: json.dumps(x))
    tx = load_rdd.map(lambda x: x.replace(",", ":").split(":")) 
    datamap = tx.map(lambda x: (["customerid",str(x[1]),"transaction_complete_dtm",str(x[3])]))
    datamap= datamap.toDF(schema=['col0', 'col1'])
    datamap.write.options(catalog=catalog).format("org.apache.spark.sql.execution.datasources.hbase").save() 

    ssc.start()
    ssc.awaitTerminationOrTimeout(15000)

if __name__ == "__main__":
    Iversondoingsomethingsilly()

#def streamrdd_to_df(srdd):
#    sdf = customparksession.createDataFrame(srdd,schema)
#    sdf.show(n=50, truncate=False)
#    sdf.write.options(catalog=catalog).format("org.apache.spark.sql.execution.datasources.hbase").save()
#    return sdf